# Dynamic RAG with Chroma and Hugging Face
Copyright 2024, Denis Rothman

The goal of this notebook is to illustrate Dynamic RAG. Dynamic RAG leverages opensource, cost-effective, lightweight temporary vector storage(Chroma) and real-time interactions (Chroma and Hugging Face Llama) with the embedded data.

[Reference: Chroma docmentation](https://docs.trychroma.com/getting-started)



# Installing the environment

## Hugging Face

Sign up on Hugging Face to obtain your  Hugging Face API token:

https://huggingface.co/

You will need it to access a Llama model and it is recommended for Hugging Face datasets.

You can use the two code options below to intialize your HF token in this program. If you are using Google Colab, you can also create a Google Secret in the sidebar and activate it. If so, you can comment the cell below.


In [1]:
# Save your Hugging Face token in a secure location

#1.Uncomment the following lines if you want to use Google Drive to retrieve your token

from google.colab import drive
drive.mount('/content/drive')
f = open("drive/MyDrive/files/hf_token.txt", "r")
access_token=f.readline().strip()
f.close()

#2.Uncomment the following line if you want to enter your HF token manually
#access_token =[YOUR HF_TOKEN]

import os
os.environ['HF_TOKEN'] = access_token

Mounted at /content/drive


In [2]:
!pip install datasets==2.20.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12

In [3]:
!pip install transformers==4.41.2

Accelerate is a library that makes it easy to run PyTorch models on multiple GPUs, TPUs, and CPUs. It also supports mixed precision training, speeds up processing times.


In [4]:
!pip install accelerate==0.31.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 9.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [5]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

## Chroma

In [6]:
!pip install chromadb==0.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 92.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 90.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 13.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 

You may need to restart the session after installing spaCy. You can try to continue without restarting the session and only restart if necessary.

In [7]:
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 35.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# Activating session time


Session time is there to measure the whole one run dynamic RAG process to prepare a daily meeting in the notebook's scenario.

It is recommended to use a GPU if one is available.

This does not include the environment installation time since this program can run on a pre-installed local machine.

In [8]:
import time
# Start timing before the request
session_start_time = time.time()

# Downloading and preparing the dataset

In [9]:
# Import required libraries
from datasets import load_dataset
import pandas as pd

# Load the SciQ dataset from HuggingFace
dataset = load_dataset("sciq", split="train")

# Filter the dataset to include only questions with support and correct answer
filtered_dataset = dataset.filter(lambda x: x["support"] != "" and x["correct_answer"] != "")


# Print the number of questions with support
print("Number of questions with support: ", len(filtered_dataset))

Generating train split:   0%|          | 0/11679 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/11679 [00:00<?, ? examples/s]

Number of questions with support:  10481


In [10]:
# Convert the filtered dataset to a pandas DataFrame
df = pd.DataFrame(filtered_dataset)

# Columns to drop
columns_to_drop = ['distractor3', 'distractor1', 'distractor2']

# Dropping the columns from the DataFrame
df.drop(columns=columns_to_drop, inplace=True)

# Create a new column 'completion' by merging 'correct_answer' and 'support'
df['completion'] = df['correct_answer'] + " because " + df['support']

# Ensure no NaN values are in the 'completion' column
df.dropna(subset=['completion'], inplace=True)
df

,question,correct_answer,support,completion
0,What type of organism is commonly used in prep...,mesophilic organisms,"Mesophiles grow best in moderate temperature, ...",mesophilic organisms because Mesophiles grow b...
1,What phenomenon makes global winds blow northe...,coriolis effect,Without Coriolis Effect the global winds would...,coriolis effect because Without Coriolis Effec...
2,Changes from a less-ordered state to a more-or...,exothermic,Summary Changes of state are examples of phase...,exothermic because Summary Changes of state ar...
3,What is the least dangerous radioactive decay?,alpha decay,All radioactive decay is dangerous to living t...,alpha decay because All radioactive decay is d...
4,Kilauea in hawaii is the world’s most continuo...,smoke and ash,Example 3.5 Calculating Projectile Motion: Hot...,smoke and ash because Example 3.5 Calculating ...
...,...,...,...,...
10476,The enzyme pepsin plays an important role in t...,peptides,Protein A large part of protein digestion take...,peptides because Protein A large part of prote...
10477,What remains a constant of radioactive substan...,rate of decay,The rate of decay of a radioactive substance i...,rate of decay because The rate of decay of a r...
10478,"Terrestrial ecosystems, also known for their d...",biomes,"Terrestrial ecosystems, also known for their d...","biomes because Terrestrial ecosystems, also kn..."
10479,High explosives create shock waves that exceed...,supersonic,The modern day formulation of gun powder is ca...,supersonic because The modern day formulation ...


In [11]:
df.shape

(10481, 4)

In [12]:
# Assuming 'df' is your DataFrame
print(df.columns)

Index(['question', 'correct_answer', 'support', 'completion'], dtype='object')


# Embedding and upserting the data in a Chroma collection



## Creating the Chroma collection

In [13]:
# Import Chroma and instantiate a client. The default Chroma client is ephemeral, meaning it will not save to disk.
import chromadb

client = chromadb.Client()

In [14]:
collection_name="sciq_supports6"

In [15]:
# List all collections
collections = client.list_collections()

# Check if the specific collection exists
collection_exists = any(collection.name == collection_name for collection in collections)
print("Collection exists:", collection_exists)

Collection exists: False


In [16]:
# Create a new Chroma collection to store the supporting evidence. We don't need to specify an embedding fuction, and the default will be used.
if collection_exists!=True:
  collection = client.create_collection(collection_name)
else:
  print("Collection ", collection_name," exists:", collection_exists)

In [17]:
# Printing the dictionary
results = collection.get()
for result in results:
    print(result)  # This will print the dictionary for each item

ids
embeddings
metadatas
documents
uris
data
included


## Selecting a model

In [18]:
model_name = "all-MiniLM-L6-v2"  # The name of the model to use for embedding

## Embedding and storing the  completions


In [19]:
ldf=len(df)

In [20]:
nb=ldf  # number of questions to embed and store
import time
start_time = time.time()  # Start timing before the request

# Convert Series to list of strings
completion_list = df["completion"][:nb].astype(str).tolist()

# Avoiding trying to load data twice in this one run dynamic RAG notebook
if collection_exists!=True:
  # Embed and store the first nb supports for this demo
  collection.add(
      ids=[str(i) for i in range(0, nb)],  # IDs are just strings
      documents=completion_list,
      metadatas=[{"type": "completion"} for _ in range(0, nb)],
  )

response_time = time.time() - start_time  # Measure response time
print(f"Response Time: {response_time:.2f} seconds")  # Print response time

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:11<00:00, 7.37MiB/s]


Response Time: 220.94 seconds


## Displaying the embeddings and the completions

In [21]:
# Fetch the collection with embeddings included
result = collection.get(include=['embeddings'])

# Extract the first embedding from the result
first_embedding = result['embeddings'][0]

# If you need to work with the length or manipulate the first embedding:
embedding_length = len(first_embedding)

print("First embedding:", first_embedding)
print("Embedding length:", embedding_length)

First embedding: [0.03689068928360939, -0.05881563201546669, -0.04818134009838104, 0.06923317164182663, 0.016696510836482048, -0.04075369983911514, 0.01883998140692711, 0.018102338537573814, 0.01780514232814312, 0.07787054777145386, 0.025281669571995735, -0.15792308747768402, -0.023618169128894806, 0.09529947489500046, -0.005831797607243061, -0.009351714514195919, 0.08793967962265015, -0.029782576486468315, -0.03175964206457138, 0.00035847260733135045, 0.04816022142767906, 0.03594561666250229, -0.06368855386972427, -0.03580130264163017, 0.008479448035359383, -0.04704919457435608, -0.014411594718694687, 0.015326135791838169, -0.017449261620640755, 0.03771507740020752, -0.05390029773116112, 0.0012937913415953517, 0.1407582312822342, -0.012112578377127647, 0.016001133248209953, 0.025889603421092033, 0.009293299168348312, -0.1314585655927658, 0.04734911024570465, 0.05548204481601715, -0.025027241557836533, 0.044910937547683716, 0.06075533106923103, -0.0013118955539539456, -0.02816570363938

In [22]:
# Fetch the collection with embeddings included
result = collection.get(include=['documents'])

# Extract the first embedding from the result
first_doc = result['documents'][0]

print("First document:", first_doc)

First document: mesophilic organisms because Mesophiles grow best in moderate temperature, typically between 25°C and 40°C (77°F and 104°F). Mesophiles are often found living in or on the bodies of humans or other animals. The optimal growth temperature of many pathogenic mesophiles is 37°C (98°F), the normal human body temperature. Mesophilic organisms have important uses in food preparation, including cheese, yogurt, beer and wine.


# Querying the collection

In [23]:
import time
start_time = time.time()  # Start timing before the request

# number of retrievals to write
results = collection.query(
    query_texts=df["question"][:nb],
    n_results=1)

response_time = time.time() - start_time  # Measure response time
print(f"Response Time: {response_time:.2f} seconds")  # Print response time

Response Time: 197.12 seconds


creating a similarity measurement function

In [24]:
import spacy
import numpy as np

# Load the pre-trained spaCy language model
nlp = spacy.load('en_core_web_md')  # Ensure that you've installed this model with 'python -m spacy download en_core_web_md'

def simple_text_similarity(text1, text2):
    # Convert the texts into spaCy document objects
    doc1 = nlp(text1)
    doc2 = nlp(text2)

    # Get the vectors for each document
    vector1 = doc1.vector
    vector2 = doc2.vector

    # Compute the cosine similarity between the two vectors
    # Check for zero vectors to avoid division by zero
    if np.linalg.norm(vector1) == 0 or np.linalg.norm(vector2) == 0:
        return 0.0  # Return zero if one of the texts does not have a vector representation
    else:
        similarity = np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))
        return similarity

displaying the query questions along with their retrieved completions, the original documents, and a simple text similarity score.

In [25]:
nbqd = 100  # the number of responses to display supposing there are more than 100 records

# Print the question, the original completion, the retrieved document, and compare them
acc_counter=0
display_counter=0
for i, q in enumerate(df['question'][:nb]):
    original_completion = df['completion'][i]  # Access the original completion for the question
    retrieved_document = results['documents'][i][0]  # Retrieve the corresponding document
    similarity_score = simple_text_similarity(original_completion, retrieved_document)
    if similarity_score > 0.7:
      acc_counter+=1
    display_counter+=1
    if display_counter<=nbqd or display_counter>nb-nbqd:
      print(i," ", f"Question: {q}")
      print(f"Retrieved document: {retrieved_document}")
      print(f"Original completion: {original_completion}")
      print(f"Similarity Score: {similarity_score:.2f}")
      print()  # Blank line for better readability between entries

if nb>0:
  acc=acc_counter/nb
  print(f"Number of documents: {nb:.2f}")
  print(f"Overall similarity score: {acc:.2f}")

0   Question: What type of organism is commonly used in preparation of foods such as cheese and yogurt?
Retrieved document: enzymes because Ingestive protists ingest, or engulf, bacteria and other small particles. They extend their cell wall and cell membrane around the food item, forming a food vacuole. Then enzymes digest the food in the vacuole.
Original completion: mesophilic organisms because Mesophiles grow best in moderate temperature, typically between 25°C and 40°C (77°F and 104°F). Mesophiles are often found living in or on the bodies of humans or other animals. The optimal growth temperature of many pathogenic mesophiles is 37°C (98°F), the normal human body temperature. Mesophilic organisms have important uses in food preparation, including cheese, yogurt, beer and wine.
Similarity Score: 0.75

1   Question: What phenomenon makes global winds blow northeast to southwest or the reverse in the northern hemisphere and northwest to southeast or the reverse in the southern hemis

# Prompt and retrieval

**Question:**

Millions of years ago, plants used energy from the sun to form what?

**Retrieved support:**

 glucose because Cellular respiration and photosynthesis are direct opposite reactions. Energy from the sun enters a plant and is converted into glucose during photosynthesis. Some of the energy is used to make ATP in the mitochondria during cellular respiration, and some is lost to the environment as heat.

In [43]:
# initial question
#prompt = "Millions of years ago, plants used energy from the sun to form what?"
# variant 1 similar
prompt = "Eons ago, plants used energy from the sun to form what?"
# variant 2 divergent
#prompt = "Eons ago, plants used sun energy to form what?"

In [44]:
import time
import textwrap

# Start timing before the request
start_time = time.time()

# Query the collection using the prompt
results = collection.query(
    query_texts=[prompt],  # Use the prompt in a list as expected by the query method
    n_results=1  # Number of results to retrieve
)

# Measure response time
response_time = time.time() - start_time

# Print response time
print(f"Response Time: {response_time:.2f} seconds\n")

# Check if documents are retrieved
if results['documents'] and len(results['documents'][0]) > 0:
    # Use textwrap to format the output for better readability
    wrapped_question = textwrap.fill(prompt, width=70)  # Wrap text at 70 characters
    wrapped_document = textwrap.fill(results['documents'][0][0], width=70)

    # Print formatted results
    print(f"Question: {wrapped_question}")
    print("\n")
    print(f"Retrieved document: {wrapped_document}")
    print()
else:
    print("No documents retrieved.")


Response Time: 0.03 seconds

Question: Eons ago, plants used energy from the sun to form what?


Retrieved document: the sun because Most of the energy used by living things comes either
directly or indirectly from the sun. That’s because sunlight provides
the energy for photosynthesis. This is the process in which plants and
certain other organisms synthesize glucose (C 6 H 12 O 6 ). The
process uses carbon dioxide and water and also produces oxygen. The
overall chemical equation for photosynthesis is:.



# RAG with Hugging Face

# RAG with Llama

In [45]:
def LLaMA2(prompt):
    sequences = pipeline(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=100,  # Control the output length more granularly
        temperature=0.5,  # Slightly higher for more diversity
        repetition_penalty=2.0,  # Adjust based on experimentation
        truncation=True
    )
    return sequences

In [46]:
iprompt='Read the following input and write a summary for beginners.'
lprompt=iprompt + " " + results['documents'][0][0]

In [47]:
import time
start_time = time.time()  # Start timing before the request

aug_prompt=lprompt +"\n"
response=LLaMA2(aug_prompt)
for seq in response:
    generated_part = seq['generated_text'].replace(iprompt, '')  # Remove the input part from the output
    print(f"Output: {generated_part.strip()}")  # .strip() to remove leading/trailing whitespace

response_time = time.time() - start_time  # Measure response time
print(f"Response Time: {response_time:.2f} seconds")  # Print response time

Output: the sun because Most of the energy used by living things comes either directly or indirectly from the sun. That’s because sunlight provides the energy for photosynthesis. This is the process in which plants and certain other organisms synthesize glucose (C 6 H 12 O 6 ). The process uses carbon dioxide and water and also produces oxygen. The overall chemical equation for photosynthesis is:.
Chemical Equation For Photosythesis Of Glcne From Carbon Dioxi Ideal Chemistry School In this article, we will discuss about photo Synthesiss anabolic steroid that promotes muscle growth without any side effects! Learn more today at our official website now.....
Response Time: 3.86 seconds


In [48]:
wrapped_response = textwrap.fill(generated_part.strip(), width=70)
print(wrapped_response)

the sun because Most of the energy used by living things comes either
directly or indirectly from the sun. That’s because sunlight provides
the energy for photosynthesis. This is the process in which plants and
certain other organisms synthesize glucose (C 6 H 12 O 6 ). The
process uses carbon dioxide and water and also produces oxygen. The
overall chemical equation for photosynthesis is:. Chemical Equation
For Photosythesis Of Glcne From Carbon Dioxi Ideal Chemistry School In
this article, we will discuss about photo Synthesiss anabolic steroid
that promotes muscle growth without any side effects! Learn more today
at our official website now.....


# Deleting the collection

Set `delete_collection=True` when the daily session is over

In [32]:
delete_collection=False
if delete_collection==True:
  client.delete_collection(collection_name)

In [33]:
# List all collections
collections = client.list_collections()

# Check if the specific collection exists
collection_exists = any(collection.name == collection_name for collection in collections)
print("Collection exists:", collection_exists)

Collection exists: True


# Total session time

Does not include environment installation time.

In [34]:
end_time = time.time() - session_start_time  # Measure response time
print(f"Session preparation time: {end_time:.2f} seconds")  # Print response time

Session preparation time: 771.10 seconds
